In [3]:
import cv2
import numpy as np
import os
import mediapipe as mp
import json # 追加: jsonモジュールをインポート

# 設定 (変更なし)
DATA_PATH = os.path.join(os.getcwd(), 'MP_Data_JSON') # 保存先ディレクトリ名を変更
# actions = np.array(['DrinkingWater', 'PushUps', 'Running', 'Squats', 'Walking', 'Standing'])
actions = np.array(['heel_hook', 'deadpoint', 'dyno','cross_move']) # こちらを使用すると仮定
no_videos = 8  # 各アクションの動画数
sequence_length = 180  # 各動画のフレーム数 (1シーケンスあたりのフレーム数)

# Mediapipe Holisticモデルの初期化 (変更なし)
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def multiple_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    # 姿勢のキーポイントのみを描画
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=1),  # ランドマーク（点）のスタイル: 青色
                             mp_drawing.DrawingSpec(color=(51, 255, 51), thickness=2, circle_radius=1)) # コネクション（線）のスタイル: 緑色

    # --- 以下をコメントアウト ---
    # 顔の描画は行わない
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
    #                          mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
    #                          mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    # 左手の描画は行わない
    # mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
    #                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=1),
    #                          mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=1))
    # 右手の描画は行わない
    # mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
    #                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=1),
    #                          mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=1))

# ★★★ 変更点: JSON形式でキーポイントを抽出 ★★★
def extract_keypoints(results):
    keypoints_data = {}

    if results.pose_landmarks:
        pose_landmarks = []
        for i, res in enumerate(results.pose_landmarks.landmark):
            pose_landmarks.append({
                "id": i,
                "x": res.x,
                "y": res.y,
                "z": res.z,
                "visibility": res.visibility
            })
        keypoints_data["pose"] = pose_landmarks
    else:
        keypoints_data["pose"] = [] # 空のリストとして保持

    if results.face_landmarks:
        face_landmarks = []
        for i, res in enumerate(results.face_landmarks.landmark):
            face_landmarks.append({
                "id": i,
                "x": res.x,
                "y": res.y,
                "z": res.z
            })
        keypoints_data["face"] = face_landmarks
    else:
        keypoints_data["face"] = []

    if results.left_hand_landmarks:
        lh_landmarks = []
        for i, res in enumerate(results.left_hand_landmarks.landmark):
            lh_landmarks.append({
                "id": i,
                "x": res.x,
                "y": res.y,
                "z": res.z
            })
        keypoints_data["left_hand"] = lh_landmarks
    else:
        keypoints_data["left_hand"] = []

    if results.right_hand_landmarks:
        rh_landmarks = []
        for i, res in enumerate(results.right_hand_landmarks.landmark):
            rh_landmarks.append({
                "id": i,
                "x": res.x,
                "y": res.y,
                "z": res.z
            })
        keypoints_data["right_hand"] = rh_landmarks
    else:
        keypoints_data["right_hand"] = []

    return keypoints_data # 辞書を返す

# データの収集
# OpenCVのウィンドウ表示設定 (変更なし)
cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)
cv2.resizeWindow('OpenCV Feed', (1280, 720))

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        action_path = os.path.join(DATA_PATH, action) # アクションごとのディレクトリパス
        if not os.path.exists(action_path):
            os.makedirs(action_path)

        for video_num in range(1, no_videos + 1):
            video_file_path = os.path.join(r'C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos', action + str(video_num) + '.mp4') # 元の動画ファイルパス
            cap = cv2.VideoCapture(video_file_path)

            # 各動画(video_num)のデータを保存するディレクトリを作成
            video_data_save_path = os.path.join(action_path, str(video_num))
            if not os.path.exists(video_data_save_path):
                os.makedirs(video_data_save_path)

            print(f'Processing video: {video_file_path}')
            frames_collected_for_current_video = 0 # この動画から収集したフレーム数をカウント

            # 1つの動画から sequence_length (180) フレームを抽出 (1回だけ)
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                if not ret:
                    print(f"Warning: Video {video_file_path} ended before collecting {sequence_length} frames. Collected {frame_num} frames.")
                    break

                image, results = multiple_detection(frame, holistic)

                draw_styled_landmarks(image, results)

                # 収集状況を画面に表示 (変更なし)
                cv2.putText(image, f'Collecting frames for {action} - Video {video_num} - Frame {frame_num + 1}/{sequence_length}',
                            (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                # ★★★ 変更点: JSONファイルとして保存 ★★★
                json_path = os.path.join(video_data_save_path, f'{frame_num}.json') # .json 拡張子に変更
                with open(json_path, 'w') as f:
                    json.dump(keypoints, f, indent=4) # 見やすいようにindent=4を指定

                frames_collected_for_current_video +=1

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            cap.release()
            if frames_collected_for_current_video < sequence_length:
                 print(f"Warning: For video {video_file_path}, only {frames_collected_for_current_video}/{sequence_length} frames were saved to {video_data_save_path}.")

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook1.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook2.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook3.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook4.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook5.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook6.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook7.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos